# Описание
- Оцениваем расхождение модельных значений коэффициентов и коэффициентов из линии по разным букмекерам
- Входной файл - dwh_pre.csv

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import importlib

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

pd.set_option('display.max_columns', 200)
plt.style.use('ggplot')

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

sys.path.append('/Users/i.tolkachev/freelance/git/softpro/')
import odds

from tqdm import tqdm_notebook

tqdm_notebook().pandas()

In [3]:
data = pd.read_csv('/Users/i.tolkachev/freelance/softpro/dwh_pre.csv')

In [4]:
data.shape

(3967436, 11)

In [5]:
data.head()

,Unnamed: 0,QueueTime,InternalID,StartTime,BookmakerName,LineKindName,IntervalKindName,MarketKindName,SelectionKindName,MarketValue,Price
0,0,2019-12-01 12:00:02,15e320bd-4f98-4d27-a761-4a9cf1b0e93a,2019-12-01 15:30:00,bookmaker0,PreMatch,FullTime,Handicap,Home,-0.25,3.150
1,1,2019-12-01 12:00:02,15e320bd-4f98-4d27-a761-4a9cf1b0e93a,2019-12-01 15:30:00,bookmaker0,PreMatch,FullTime,Handicap,Away,-0.25,1.395
2,2,2019-12-01 12:00:02,15e320bd-4f98-4d27-a761-4a9cf1b0e93a,2019-12-01 15:30:00,bookmaker0,PreMatch,FullTime,Handicap,Home,-0.50,3.640
3,3,2019-12-01 12:00:02,15e320bd-4f98-4d27-a761-4a9cf1b0e93a,2019-12-01 15:30:00,bookmaker0,PreMatch,FullTime,Handicap,Away,-0.50,1.315
4,4,2019-12-01 12:00:02,15e320bd-4f98-4d27-a761-4a9cf1b0e93a,2019-12-01 15:30:00,bookmaker0,PreMatch,FullTime,Handicap,Home,0.25,2.090


In [6]:
print('Уникальных матчей в файле: ', data.InternalID.nunique())

Уникальных матчей в файле:  618


In [8]:
def create_odds(m_k_name, s_k_name, m_val, price):
    return dict(
        zip(
            m_k_name + ' ' + s_k_name + ' ' + m_val.astype(str).tolist(),
            price.tolist()
        )
    )

In [9]:
# создаем словарь с кэфами для каждого матча, по каждому букмекеру, в каждый момент времени
data_odds_dict = (
    data
    .groupby(['QueueTime', 'InternalID', 'StartTime', 'BookmakerName', 'LineKindName', 'IntervalKindName'])
    [['MarketKindName', 'SelectionKindName', 'MarketValue', 'Price']]
    .progress_apply(lambda x:
        create_odds(x['MarketKindName'], x['SelectionKindName'], x['MarketValue'], x['Price'])
    )
)

In [11]:
data.groupby('InternalID')['MarketKindName'].count().head()\

InternalID
00e3c7b3-ad36-447a-82b6-3dd1860ba1a3     4009
010cdc53-3c55-47dc-b99b-eabf9ddc4854      957
011c00aa-e567-4425-aed8-6dab1d08d137    31898
01a826ea-805f-4851-bc39-b1c9711148f2     4095
01c77461-34c9-4f70-9f9a-9a779b5a296c    30856
Name: MarketKindName, dtype: int64

In [13]:
data['MarketKindName'].unique()

array(['Handicap', 'Total', 'ThreeWay', 'HomeTeamTotal', 'AwayTeamTotal',
       'BothTeamToScore', 'HomeToWinToNil', 'AwayToWinToNil'],
      dtype=object)

In [14]:
data[data['MarketKindName'] == 'HomeTeamTotal'].head()

,Unnamed: 0,QueueTime,InternalID,StartTime,BookmakerName,LineKindName,IntervalKindName,MarketKindName,SelectionKindName,MarketValue,Price
130,204,2019-12-01 12:00:20,e89ce591-7ec4-44f1-b30f-b6f7cdacb1e7,2019-12-01 19:00:00,bookmaker4,PreMatch,FullTime,HomeTeamTotal,Over,1.5,1.704
131,205,2019-12-01 12:00:20,e89ce591-7ec4-44f1-b30f-b6f7cdacb1e7,2019-12-01 19:00:00,bookmaker4,PreMatch,FullTime,HomeTeamTotal,Under,1.5,2.180
163,237,2019-12-01 12:00:20,e8cc72bd-48b8-40ed-a06c-e6d7eb1cfe8d,2019-12-01 14:00:00,bookmaker4,PreMatch,FullTime,HomeTeamTotal,Over,1.5,1.813
164,238,2019-12-01 12:00:20,e8cc72bd-48b8-40ed-a06c-e6d7eb1cfe8d,2019-12-01 14:00:00,bookmaker4,PreMatch,FullTime,HomeTeamTotal,Under,1.5,2.070
363,546,2019-12-01 12:00:40,e89ce591-7ec4-44f1-b30f-b6f7cdacb1e7,2019-12-01 19:00:00,bookmaker4,PreMatch,FullTime,HomeTeamTotal,Over,1.5,1.675


In [15]:
data_odds_dict = data_odds_dict.reset_index()

In [16]:
data_odds_dict.head()

,QueueTime,InternalID,StartTime,BookmakerName,LineKindName,IntervalKindName,0
0,2019-12-01 12:00:02,15e320bd-4f98-4d27-a761-4a9cf1b0e93a,2019-12-01 15:30:00,bookmaker0,PreMatch,FullTime,"{'Handicap Home -0.25': 3.15, 'Handicap Away -..."
1,2019-12-01 12:00:02,8a143bfb-1700-4d3c-b2d8-480d7f2f8f9d,2019-12-01 15:30:00,bookmaker0,PreMatch,FullTime,"{'Handicap Home -0.5': 3.91, 'Handicap Away -0..."
2,2019-12-01 12:00:05,91fd064b-5ecc-4e9c-af3f-0a258e5a9394,2019-12-01 16:30:00,bookmaker1,PreMatch,FullTime,"{'Handicap Home -2.0': 2.36, 'Handicap Away -2..."
3,2019-12-01 12:00:08,0aa80a7e-1613-41e3-af78-f1d26382e386,2019-12-01 18:00:00,bookmaker2,PreMatch,FullTime,"{'Total Over 2.25': 1.88, 'Total Under 2.25': ..."
4,2019-12-01 12:00:08,26aab835-071d-40a8-bc53-86baf36037bc,2019-12-01 16:30:00,bookmaker2,PreMatch,FullTime,"{'ThreeWay Home 0.0': 4.8, 'ThreeWay Away 0.0'..."


In [17]:
# get pinnacle column names
# file 20191125.xlsx
data_3 = pd.read_excel('/Users/i.tolkachev/freelance/softpro/20191125.xlsx', sheet_name='Sheet2')

columns_data = [c for c in data_3.columns.tolist() if not ('computed' in c or 'disc' in c)]

ml_columns = ['HOME', 'DRAW', 'AWAY']
total_columns = [col for col in columns_data if 'TO' in col or 'TU' in col]
total_home_columns = [col for col in columns_data if 'T1O' in col or 'T1U' in col]
total_away_columns = [col for col in columns_data if 'T2O' in col or 'T2U' in col]
hdp_columns = [col for col in columns_data if 'HDP' in col]

In [18]:
# map column formats (pinnacle --- others)
map_col = {}
map_col_inv = {}
for c in hdp_columns:
    cur = 'Handicap'
    if c[:4] == 'HOME':
        cur += ' Home'
    else:
        cur += ' Away'
    hdp_val = c.split(' ')[1].replace(',', '.')
    cur += f' {hdp_val}'
    map_col[c] = cur
    map_col_inv[cur] = c
    
for c in total_columns:
    cur = 'Total'
    if c[1] == 'O':
        cur += ' Over'
    else:
        cur += ' Under'
    hdp_val = c[3:].replace('_', '.')
    cur += f' {hdp_val}'
    map_col[c] = cur
    map_col_inv[cur] = c
    
for c in total_home_columns:
    cur = 'HomeTeamTotal'
    if c[2] == 'O':
        cur += ' Over'
    else:
        cur += ' Under'
    hdp_val = c[4:].replace('_', '.')
    cur += f' {hdp_val}'
    map_col[c] = cur
    map_col_inv[cur] = c

for c in total_away_columns:
    cur = 'AwayTeamTotal'
    if c[2] == 'O':
        cur += ' Over'
    else:
        cur += ' Under'
    hdp_val = c[4:].replace('_', '.')
    cur += f' {hdp_val}'
    map_col[c] = cur
    map_col_inv[cur] = c

for c in ml_columns:
    cur = 'ThreeWay'
    if c == 'HOME':
        cur += ' Home'
    elif c == 'DRAW':
        cur += ' Draw'
    else:
        cur += ' Away'
    cur += f' 0.0'
    map_col[c] = cur
    map_col_inv[cur] = c

In [19]:
def map_col_names(x):
    ans = {}
    for k,v in x.items():
        key = map_col_inv.get(k)
        if key:
            ans[map_col_inv.get(k)] = v
    return ans

In [21]:
data_odds_dict['pinnacle_odds'] = data_odds_dict[0].progress_apply(map_col_names)

In [22]:
book_odds = data_odds_dict['pinnacle_odds'].progress_apply(pd.Series)

In [23]:
book_odds[['QueueTime', 'InternalID', 'StartTime', 'BookmakerName', 'LineKindName']] = (
    data_odds_dict[['QueueTime', 'InternalID', 'StartTime', 'BookmakerName', 'LineKindName']]
)

In [24]:
book_odds.head(1)

,"HOME_HDP -0,25","AWAY_HDP -0,25","HOME_HDP -0,5","AWAY_HDP -0,5","HOME_HDP 0,25","AWAY_HDP 0,25","HOME_HDP 1,5","AWAY_HDP 1,5","HOME_HDP 0,75","AWAY_HDP 0,75","HOME_HDP 0,5","AWAY_HDP 0,5","HOME_HDP 0,0","AWAY_HDP 0,0",TO_2_75,TU_2_75,TO_2_0,TU_2_0,TO_3_0,TU_3_0,TO_3_5,TU_3_5,TO_4_5,TU_4_5,TO_2_5,TU_2_5,TO_1_5,TU_1_5,TO_2_25,TU_2_25,HOME,AWAY,DRAW,"HOME_HDP 1,0","AWAY_HDP 1,0","HOME_HDP -2,0","AWAY_HDP -2,0","AWAY_HDP 2,25","HOME_HDP -0,75","AWAY_HDP -0,75","HOME_HDP -1,25","HOME_HDP -1,0","AWAY_HDP -1,0","HOME_HDP -1,5","AWAY_HDP -1,5","HOME_HDP -2,5","AWAY_HDP -2,5",TO_3_25,TU_3_25,T1O_1_5,T1U_1_5,T2O_1_0,T2U_1_0,TO_1_75,TU_1_75,T2O_0_5,T2U_0_5,"AWAY_HDP -1,75","AWAY_HDP 1,25","HOME_HDP 3,5","AWAY_HDP 3,5","HOME_HDP 2,75","AWAY_HDP 2,75","HOME_HDP 2,5","AWAY_HDP 2,5","HOME_HDP 2,0","AWAY_HDP 2,0","HOME_HDP 1,75",TO_4_0,TU_4_0,TO_6_5,TU_6_5,TO_5_5,TU_5_5,TO_4_25,TU_4_25,TO_3_75,TU_3_75,T1O_1_0,T1U_1_0,T2O_3_5,T2U_3_5,T2O_1_5,T2U_1_5,"HOME_HDP -2,25",T2O_2_5,T2U_2_5,"HOME_HDP -3,5","AWAY_HDP -3,5",T1O_2_5,T1U_2_5,T2O_3_0,T2U_3_0,T1O_2_0,T1U_2_0,T1O_0_5,T1U_0_5,"HOME_HDP -2,75","AWAY_HDP -2,75",T1O_3_5,T1U_3_5,TO_1_25,TU_1_25,TO_0_5,TU_0_5,T2O_2_0,T2U_2_0,"HOME_HDP -3,0","AWAY_HDP -3,0",T1O_3_0,T1U_3_0,T2U_4_5,T2O_4_5,"HOME_HDP -3,25","AWAY_HDP 3,25","HOME_HDP 3,0","AWAY_HDP 3,0",TO_4_75,TU_4_75,TO_1_0,TU_1_0,TO_0_75,TU_0_75,TO_5_0,TU_5_0,TO_5_25,TU_5_25,T1U_4_5,T1O_4_5,TO_5_75,TU_5_75,TO_6_0,TU_6_0,TO_6_25,TU_6_25,QueueTime,InternalID,StartTime,BookmakerName,LineKindName
0,3.15,1.395,3.64,1.315,2.09,1.813,1.261,4.12,1.591,2.47,1.8,2.1,2.66,1.523,2.08,1.806,1.373,3.21,2.48,1.578,3.16,1.381,6.21,1.137,1.847,2.03,1.239,4.26,1.606,2.41,3.64,2.1,3.51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-12-01 12:00:02,15e320bd-4f98-4d27-a761-4a9cf1b0e93a,2019-12-01 15:30:00,bookmaker0,PreMatch


In [25]:
# order by QueueTime
book_odds_ordered = book_odds.sort_values(by=['InternalID', 'BookmakerName', 'QueueTime'])

In [33]:
book_odds_ordered.head(1)

,"HOME_HDP -0,25","AWAY_HDP -0,25","HOME_HDP -0,5","AWAY_HDP -0,5","HOME_HDP 0,25","AWAY_HDP 0,25","HOME_HDP 1,5","AWAY_HDP 1,5","HOME_HDP 0,75","AWAY_HDP 0,75","HOME_HDP 0,5","AWAY_HDP 0,5","HOME_HDP 0,0","AWAY_HDP 0,0",TO_2_75,TU_2_75,TO_2_0,TU_2_0,TO_3_0,TU_3_0,TO_3_5,TU_3_5,TO_4_5,TU_4_5,TO_2_5,TU_2_5,TO_1_5,TU_1_5,TO_2_25,TU_2_25,HOME,AWAY,DRAW,"HOME_HDP 1,0","AWAY_HDP 1,0","HOME_HDP -2,0","AWAY_HDP -2,0","AWAY_HDP 2,25","HOME_HDP -0,75","AWAY_HDP -0,75","HOME_HDP -1,25","HOME_HDP -1,0","AWAY_HDP -1,0","HOME_HDP -1,5","AWAY_HDP -1,5","HOME_HDP -2,5","AWAY_HDP -2,5",TO_3_25,TU_3_25,T1O_1_5,T1U_1_5,T2O_1_0,T2U_1_0,TO_1_75,TU_1_75,T2O_0_5,T2U_0_5,"AWAY_HDP -1,75","AWAY_HDP 1,25","HOME_HDP 3,5","AWAY_HDP 3,5","HOME_HDP 2,75","AWAY_HDP 2,75","HOME_HDP 2,5","AWAY_HDP 2,5","HOME_HDP 2,0","AWAY_HDP 2,0","HOME_HDP 1,75",TO_4_0,TU_4_0,TO_6_5,TU_6_5,TO_5_5,TU_5_5,TO_4_25,TU_4_25,TO_3_75,TU_3_75,T1O_1_0,T1U_1_0,T2O_3_5,T2U_3_5,T2O_1_5,T2U_1_5,"HOME_HDP -2,25",T2O_2_5,T2U_2_5,"HOME_HDP -3,5","AWAY_HDP -3,5",T1O_2_5,T1U_2_5,T2O_3_0,T2U_3_0,T1O_2_0,T1U_2_0,T1O_0_5,T1U_0_5,"HOME_HDP -2,75","AWAY_HDP -2,75",T1O_3_5,T1U_3_5,TO_1_25,TU_1_25,TO_0_5,TU_0_5,T2O_2_0,T2U_2_0,"HOME_HDP -3,0","AWAY_HDP -3,0",T1O_3_0,T1U_3_0,T2U_4_5,T2O_4_5,"HOME_HDP -3,25","AWAY_HDP 3,25","HOME_HDP 3,0","AWAY_HDP 3,0",TO_4_75,TU_4_75,TO_1_0,TU_1_0,TO_0_75,TU_0_75,TO_5_0,TU_5_0,TO_5_25,TU_5_25,T1U_4_5,T1O_4_5,TO_5_75,TU_5_75,TO_6_0,TU_6_0,TO_6_25,TU_6_25,QueueTime,InternalID,StartTime,BookmakerName,LineKindName
182784,1.581,2.43,1.806,2.06,NaN,NaN,NaN,NaN,NaN,NaN,1.235,4.19,1.386,3.03,1.694,2.19,NaN,NaN,1.917,1.934,2.43,1.584,4.51,1.202,1.561,2.48,1.158,5.27,NaN,NaN,1.826,4.25,3.96,NaN,NaN,NaN,NaN,NaN,2.01,1.869,2.83,2.4,1.613,3.19,1.364,6.78,1.113,2.17,1.709,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-12-08 10:52:07,00e3c7b3-ad36-447a-82b6-3dd1860ba1a3,2019-12-08 19:00:00,bookmaker0,PreMatch


In [35]:
def fair_odd(odd, margin):
    return (odd*(margin+1))

def margin_2(odd_1, odd_2):
    return 1/odd_1 + 1/odd_2 - 1

def margin_3(odd_1, odd_2, odd_3):
    return 1/odd_1 + 1/odd_2 + 1/odd_3 - 1

In [47]:
2.5 % 1

0.5

In [79]:
# main function for computing errors
from collections import defaultdict

total_errors = defaultdict(list)
int_total_errors = defaultdict(list)
half_total_errors = defaultdict(list)
quarter_total_errors = defaultdict(list)

total_1_errors = defaultdict(list)
int_total_1_errors = defaultdict(list)
half_total_1_errors = defaultdict(list)
quarter_total_1_errors = defaultdict(list)

total_2_errors = defaultdict(list)
int_total_2_errors = defaultdict(list)
half_total_2_errors = defaultdict(list)
quarter_total_2_errors = defaultdict(list)

hdp_1_errors = defaultdict(list)
int_hdp_1_errors = defaultdict(list)
half_hdp_1_errors = defaultdict(list)
quarter_hdp_1_errors = defaultdict(list)

ml_errors = defaultdict(list)


for i in tqdm_notebook(book_odds_ordered[['InternalID', 'BookmakerName']].drop_duplicates().values):
    try:
        book_for_check = book_odds_ordered[
            (book_odds_ordered['InternalID'] == i[0]) & 
            (book_odds_ordered['BookmakerName'] == i[1])
        ].fillna(method='ffill')
        
        print('all match rows shape:', book_for_check.shape)
        book_for_check['not_null'] = book_for_check.notnull().sum(axis=1)
        book_for_check['max_not_null'] = (
            book_for_check
            .groupby(['InternalID', 'BookmakerName'])['not_null']
            .transform(max)
        )
        
        book_for_check['max_queue_time'] = (
            book_for_check
            .groupby(['InternalID', 'BookmakerName'])['QueueTime']
            .transform(max)
        )

        book_for_check = book_for_check[
            (book_for_check['QueueTime'] < book_for_check['StartTime'])
            & 
            (book_for_check['not_null'] == book_for_check['max_not_null'])
        ]
        
        filled_data = book_for_check.iloc[[-1], :].dropna(axis=1).reset_index(drop=True)
#         print('input odds before game started', filled_data.shape)
#         print(filled_data.head())
        
        not_empty_columns = [c for c in filled_data.columns if not ('computed' in c or 'disc' in c)]
        filled_data = filled_data[not_empty_columns]

        ml_columns = ['HOME', 'DRAW', 'AWAY']
        total_columns = [col for col in not_empty_columns if 'TO' in col or 'TU' in col]
        total_home_columns = [col for col in not_empty_columns if 'T1O' in col or 'T1U' in col]
        total_away_columns = [col for col in not_empty_columns if 'T2O' in col or 'T2U' in col]
        hdp_columns = [col for col in not_empty_columns if 'HDP' in col]
        
        # compute model init parameters
        half_total_over = [c for c in total_columns 
                           if ('TO' in c) 
                           and ('_5' in c) 
                           and (filled_data[c][0] > 1.5) 
                           and (filled_data[c][0] < 3)
                          ][0]
        half_total_value = float(half_total_over[3:].replace('_', '.'))
        half_total_init = [half_total_over, half_total_over.replace('O', 'U')]

        half_hdp_home = [c for c in hdp_columns 
                         if ('HOME_HDP' in c)
                         and (',5' in c) 
                         and (filled_data[c][0] > 1.5) 
                         and (filled_data[c][0] < 3)
                        ][0]
        half_hdp_value = float(half_hdp_home[8:].replace(',', '.'))
        half_hdp_init = [half_hdp_home, f'AWAY_HDP {half_hdp_value}'.replace('.', ',')]

        # margin
        half_total_margin = margin_2(
            *filled_data[half_total_init].values[0].tolist()
        )

        half_hdp_margin = margin_2(
            *filled_data[half_hdp_init].values[0].tolist()
        )
        
#         print('total margin:', half_total_margin)
#         print('hdp margin:', half_hdp_margin)

        to, tu = filled_data[half_total_init].values[0].tolist()
        fair_to, fair_tu = fair_odd(filled_data[half_total_init], half_total_margin).values[0].tolist()

        hdp1, hdp2 = filled_data[half_hdp_init].values[0].tolist()
        fair_hdp1, fair_hdp2 = fair_odd(filled_data[half_hdp_init], half_hdp_margin).values[0].tolist()

        print(half_total_value, to, tu, half_hdp_value, hdp1, hdp2)
        book = filled_data.iloc[0, :]['BookmakerName']
        try:
            result_odds, mu, mu1, mu2 = odds.compute_odds(half_total_value, to, tu,
                                                          half_hdp_value, hdp1, hdp2)

#             print('TO fair: ', fair_to, 'TU fair: ', fair_tu)
#             to_pred = result_odds[f'T O {half_total_value}']
#             tu_pred = result_odds[f'T U {half_total_value}']
#             print('TO pred: ', to_pred, 'TU fair: ', tu_pred)

#             plt.figure(figsize=(10, 10))
            x, y, y_pred = [], [], []
            # totals errors
            for i in range(len(total_columns[::2])):
                to, tu = filled_data[total_columns[2*i:2*i+2]].values[0].tolist()
                t_val = float(total_columns[2*i][3:].replace('_', '.'))
                margin = margin_2(to, tu)
                fair_to = fair_odd(to, margin)
                fair_tu = fair_odd(tu, margin)
#                 print('TO: ', to, 'TU: ', tu, 'margin', margin)
#                 print('T value: ', t_val)
#                 print('TO fair: ', fair_to, 'TU fair: ', fair_tu)
                to_pred = result_odds[f'T O {t_val}']
                tu_pred = result_odds[f'T U {t_val}']
#                 print('TO pred: ', to_pred, 'TU fair: ', tu_pred)
#                 print('Error TO: ', abs(odds.odd_to_prob(fair_to) - odds.odd_to_prob(to_pred)))
#                 print('Error TU: ', abs(odds.odd_to_prob(fair_tu) - odds.odd_to_prob(tu_pred)))
                x.append(t_val)
                y.append(odds.odd_to_prob(fair_tu))
                y_pred.append(odds.odd_to_prob(tu_pred))
                
                total_errors[book].append(abs(odds.odd_to_prob(fair_tu) - odds.odd_to_prob(tu_pred)))
                if t_val % 1 == 0.5:
                    half_total_errors[book].append(abs(odds.odd_to_prob(fair_tu) - odds.odd_to_prob(tu_pred)))
                elif t_val % 1 == 0.0:
                    int_total_errors[book].append(abs(odds.odd_to_prob(fair_tu) - odds.odd_to_prob(tu_pred)))
                else:
                    quarter_total_errors[book].append(abs(odds.odd_to_prob(fair_tu) - odds.odd_to_prob(tu_pred)))

#             plt.plot(x, y, 'o', label="fair TU", alpha=0.7)
#             plt.plot(x, y_pred, 'o', label="pred TU", alpha=0.7)

#             # plot poisson distr
#             x, y = [], []
#             for x0 in range(0, 7):
#                 x.append(x0+0.5)
#                 p_opt = odds.PoissonOptimizer(x0, 0.5)
#                 y.append(p_opt.sum_pmf(mu))
#             plt.plot(x, y, 'o-', label="poisson pmf", alpha=0.7)
#             plt.legend(loc='upper left')
#             plt.show()

            # totals home errors
            for i in range(len(total_home_columns[::2])):
                to, tu = filled_data[total_home_columns[2*i:2*i+2]].values[0].tolist()
                t_val = float(total_home_columns[2*i][4:].replace('_', '.'))
                margin = margin_2(to, tu)
                fair_to = fair_odd(to, margin)
                fair_tu = fair_odd(tu, margin)
#                 print('T1O: ', to, 'T1U: ', tu, 'margin', margin)
#                 print('T value: ', t_val)
#                 print('T1O fair: ', fair_to, 'T1U fair: ', fair_tu)
                to_pred = result_odds[f'T1 O {t_val}']
                tu_pred = result_odds[f'T1 U {t_val}']
#                 print('T1O pred: ', to_pred, 'T1U fair: ', tu_pred)
#                 print('Error T1O: ', abs(odds.odd_to_prob(fair_to) - odds.odd_to_prob(to_pred)))
#                 print('Error T1U: ', abs(odds.odd_to_prob(fair_tu) - odds.odd_to_prob(tu_pred)))
#                 print()
    
                total_1_errors[book].append(abs(odds.odd_to_prob(fair_tu) - odds.odd_to_prob(tu_pred)))
                if t_val % 1 == 0.5:
                    half_total_1_errors[book].append(abs(odds.odd_to_prob(fair_tu) - odds.odd_to_prob(tu_pred)))
                elif t_val % 1 == 0.0:
                    int_total_1_errors[book].append(abs(odds.odd_to_prob(fair_tu) - odds.odd_to_prob(tu_pred)))
                else:
                    quarter_total_1_errors[book].append(abs(odds.odd_to_prob(fair_tu) - odds.odd_to_prob(tu_pred)))
#                 total_1_errors.append(abs(odds.odd_to_prob(fair_tu) - odds.odd_to_prob(tu_pred)))

            # totals away errors
            for i in range(len(total_away_columns[::2])):
                to, tu = filled_data[total_away_columns[2*i:2*i+2]].values[0].tolist()
                t_val = float(total_away_columns[2*i][4:].replace('_', '.'))
                margin = margin_2(to, tu)
                fair_to = fair_odd(to, margin)
                fair_tu = fair_odd(tu, margin)
#                 print('T2O: ', to, 'T2U: ', tu, 'margin', margin)
#                 print('T value: ', t_val)
#                 print('T2O fair: ', fair_to, 'T2U fair: ', fair_tu)
                to_pred = result_odds[f'T2 O {t_val}']
                tu_pred = result_odds[f'T2 U {t_val}']
#                 print('T2O pred: ', to_pred, 'T2U fair: ', tu_pred)
#                 print('Error T2O: ', abs(odds.odd_to_prob(fair_to) - odds.odd_to_prob(to_pred)))
#                 print('Error T2U: ', abs(odds.odd_to_prob(fair_tu) - odds.odd_to_prob(tu_pred)))
#                 total_2_errors.append(abs(odds.odd_to_prob(fair_to) - odds.odd_to_prob(to_pred)))
                total_2_errors[book].append(abs(odds.odd_to_prob(fair_tu) - odds.odd_to_prob(tu_pred)))
                if t_val % 1 == 0.5:
                    half_total_2_errors[book].append(abs(odds.odd_to_prob(fair_tu) - odds.odd_to_prob(tu_pred)))
                elif t_val % 1 == 0.0:
                    int_total_2_errors[book].append(abs(odds.odd_to_prob(fair_tu) - odds.odd_to_prob(tu_pred)))
                else:
                    quarter_total_2_errors[book].append(abs(odds.odd_to_prob(fair_tu) - odds.odd_to_prob(tu_pred)))
            
            # handicap errors
            for home_col in [c for c in hdp_columns if 'HOME' in c]:
                away_col = 'AWAY' + home_col[4:]
                hdp_value = float(home_col.split(' ')[1].replace(',','.'))
#                 print(home_col, away_col, hdp_value)
                try:
                    hdp_home, hdp_away = filled_data[[home_col, away_col]].values[0].tolist()
                except:
                    print('no alternative odd')
                    continue
                margin = margin_2(hdp_home, hdp_away)
#                 print('hdp margin:', margin)
                fair_home_hdp = fair_odd(hdp_home, margin)
                fair_away_hdp = fair_odd(hdp_away, margin)
#                 print('T2O: ', to, 'T2U: ', tu, 'margin', margin)
#                 print('T value: ', t_val)
#                 print('T2O fair: ', fair_to, 'T2U fair: ', fair_tu)
                home_hdp_pred = result_odds[f'HDP1 {hdp_value}']
#                 away_hdp_pred = result_odds[f'HDP2 {(-1)*hdp_value}']
                
                hdp_1_errors[book].append(abs(odds.odd_to_prob(fair_home_hdp) - odds.odd_to_prob(home_hdp_pred)))
                if t_val % 1 == 0.5:
                    half_hdp_1_errors[book].append(abs(odds.odd_to_prob(fair_home_hdp) - odds.odd_to_prob(home_hdp_pred)))
                elif t_val % 1 == 0.0:
                    int_hdp_1_errors[book].append(abs(odds.odd_to_prob(fair_home_hdp) - odds.odd_to_prob(home_hdp_pred)))
                else:
                    quarter_hdp_1_errors[book].append(abs(odds.odd_to_prob(fair_home_hdp) - odds.odd_to_prob(home_hdp_pred)))

    
            # ML
            home, draw, away = filled_data[ml_columns].values[0].tolist()
            margin = margin_3(home, draw, away)

            print('ML :', home, draw, away)
#             print('ML margin:', margin)
            fair_home = fair_odd(home, margin)
            home_pred = result_odds[f'ML 1']
#             print('Home fair: ', fair_home, 'Home pred: ', home_pred)
#             print('Error home: ', abs(odds.odd_to_prob(fair_home) - odds.odd_to_prob(home_pred)))
            ml_errors[book].append(abs(odds.odd_to_prob(fair_home) - odds.odd_to_prob(home_pred)))

            fair_draw = fair_odd(draw, margin)
            draw_pred = result_odds[f'ML X']
#             print('Draw fair: ', fair_draw, 'Draw pred: ', draw_pred)
#             print('Error draw: ', abs(odds.odd_to_prob(fair_draw) - odds.odd_to_prob(draw_pred)))
            ml_errors[book].append(abs(odds.odd_to_prob(fair_draw) - odds.odd_to_prob(draw_pred)))
        
            fair_away = fair_odd(away, margin)
            away_pred = result_odds[f'ML 2']
#             print('Away fair: ', fair_away, 'Away pred: ', away_pred)
#             print('Error away: ', abs(odds.odd_to_prob(fair_away) - odds.odd_to_prob(away_pred)))
            ml_errors[book].append(abs(odds.odd_to_prob(fair_away) - odds.odd_to_prob(away_pred)))

#             # plot skellam distr
#             plt.figure(figsize=(10, 10))
#             x, y = [], []
#             for x0 in range(-6, 7):
#                 x.append(x0)
#                 s_opt = odds.SkellamOptimizer(x0, 0.5, mu)
#                 y.append(s_opt.pmf(x0, mu2))
#             plt.plot(x, y, 'o-', label="skellam pmf", alpha=0.7)
#             plt.scatter([0], [odds.odd_to_prob(fair_draw)], label="fair draw", alpha=0.7)
#             plt.scatter([0], [odds.odd_to_prob(draw_pred)], label="pred draw", alpha=0.7)
#             plt.legend(loc='upper left')
#             plt.show()
        except StopIteration as e:
            print(e)
    except Exception as e:
        print('next', e)

all match rows shape: (55, 140)
3.5 2.38 1.617 -0.5 1.68 2.29
no alternative odd
ML : 1.68 4.41 4.63
all match rows shape: (44, 140)
next list index out of range
all match rows shape: (86, 140)
next list index out of range
all match rows shape: (152, 140)
next list index out of range
all match rows shape: (40, 140)
3.5 2.38 1.617 -0.5 1.68 2.29
no alternative odd
ML : 1.68 4.41 4.63
all match rows shape: (51, 140)
3.5 2.35 1.575 -0.5 1.7 2.1
no alternative odd
no alternative odd
ML : 1.73 4.0 4.33
all match rows shape: (4, 140)
3.5 2.42 1.6 -0.5 1.76 2.12
no alternative odd
ML : 1.77 4.1 4.34
all match rows shape: (11, 140)
2.5 1.833 1.961 0.5 2.31 1.598
ML : 5.2 3.87 1.598
all match rows shape: (12, 140)
next list index out of range
all match rows shape: (29, 140)
next list index out of range
all match rows shape: (3, 140)
2.5 1.787 2.01 1.5 1.584 2.35
ML : 6.09 4.4 1.454
all match rows shape: (10, 140)
next list index out of range
all match rows shape: (14, 140)
2.5 1.91 1.93 0.5 2.3

ML : 3.07 3.58 2.3
all match rows shape: (52, 140)
2.5 1.934 1.952 -0.5 2.3 1.675
no alternative odd
ML : 2.3 3.43 3.25
all match rows shape: (82, 140)
2.5 1.91 1.91 -0.5 2.31 1.6
ML : 2.33 3.21 3.06
all match rows shape: (77, 140)
2.5 1.89 2.01 -0.5 2.29 1.69
ML : 2.28 3.4 3.2
all match rows shape: (102, 140)
2.5 1.91 1.99 -0.5 2.29 1.69
ML : 2.26 3.4 3.2
all match rows shape: (37, 140)
2.5 1.934 1.952 -0.5 2.3 1.675
no alternative odd
ML : 2.3 3.43 3.25
all match rows shape: (60, 140)
2.5 1.775 2.025 -0.5 2.025 1.775
no alternative odd
no alternative odd
ML : 2.0 3.6 3.5
all match rows shape: (2, 140)
2.5 1.96 1.89 -0.5 2.07 1.79
ML : 2.09 3.46 3.66
all match rows shape: (70, 140)
3.5 2.86 1.462 -0.5 2.11 1.813
ML : 2.11 3.72 3.47
all match rows shape: (140, 140)
2.5 1.7 2.21 -0.5 2.17 1.76
ML : 2.16 3.48 3.33
all match rows shape: (168, 140)
2.5 1.74 2.21 -0.5 2.16 1.81
ML : 2.06 3.65 3.5
all match rows shape: (330, 140)
2.5 1.74 2.21 -0.5 2.16 1.81
ML : 2.14 3.65 3.35
all match row

/Users/i.tolkachev/freelance/git/softpro/optimizers.py:110: RuntimeWarning: invalid value encountered in power
  return np.exp(-self.mu) * ((self.mu - mu2) / mu2) ** (k / 2) * iv(k, 2 * np.sqrt((self.mu - mu2) * mu2))
/Users/i.tolkachev/freelance/git/softpro/optimizers.py:110: RuntimeWarning: invalid value encountered in sqrt
  return np.exp(-self.mu) * ((self.mu - mu2) / mu2) ** (k / 2) * iv(k, 2 * np.sqrt((self.mu - mu2) * mu2))


all match rows shape: (19, 140)
2.5 2.2 1.729 -0.5 2.68 1.526
ML : 2.68 3.39 2.73
all match rows shape: (18, 140)
next list index out of range
all match rows shape: (72, 140)
next list index out of range
all match rows shape: (70, 140)
next list index out of range
all match rows shape: (16, 140)
2.5 2.2 1.729 -0.5 2.68 1.526
ML : 2.68 3.39 2.73
all match rows shape: (155, 140)
2.5 2.1 1.7 -0.5 2.6 1.475
no alternative odd
ML : 2.55 3.3 2.7
all match rows shape: (7, 140)
2.5 2.19 1.72 -0.5 2.66 1.51
ML : 2.71 3.24 2.77
all match rows shape: (157, 140)
4.5 2.1 1.787 -2.5 1.74 2.19
no alternative odd
no alternative odd
ML : 1.108 11.24 21.16
all match rows shape: (158, 140)
4.5 2.075 1.725 -2.5 1.775 2.025
no alternative odd
no alternative odd
no alternative odd
ML : 1.125 10.0 17.0
all match rows shape: (130, 140)
next positional indexers are out-of-bounds
all match rows shape: (13, 140)
2.5 1.854 2.03 0.5 1.6840000000000002 2.28
ML : 3.13 3.61 2.28
all match rows shape: (20, 140)
next l

no alternative odd
ML : 1.568 4.3 5.73
all match rows shape: (22, 140)
2.5 2.0 1.85 -0.5 1.6 2.3
no alternative odd
no alternative odd
ML : 1.57 4.0 5.75
all match rows shape: (3, 140)
2.5 2.01 1.84 -0.5 1.66 2.28
no alternative odd
ML : 1.67 3.84 5.51
all match rows shape: (245, 140)
2.5 2.32 1.662 -0.5 2.76 1.505
ML : 2.76 3.1 2.91
all match rows shape: (198, 140)
2.5 2.25 1.64 -0.5 2.66 1.47
ML : 2.74 2.9 2.89
all match rows shape: (230, 140)
next list index out of range
all match rows shape: (500, 140)
2.5 2.31 1.68 -0.5 2.75 1.52
ML : 2.72 2.99 2.96
all match rows shape: (91, 140)
2.5 2.32 1.662 -0.5 2.77 1.5
ML : 2.77 3.1 2.9
all match rows shape: (188, 140)
2.5 2.15 1.675 -0.5 2.375 1.55
no alternative odd
no alternative odd
ML : 2.55 3.25 2.8
all match rows shape: (45, 140)
next positional indexers are out-of-bounds
all match rows shape: (88, 140)
3.5 2.47 1.578 -0.5 1.526 2.65
no alternative odd
ML : 1.526 4.57 5.94
all match rows shape: (108, 140)
next list index out of range

...

all match rows shape: (70, 140)
next list index out of range
all match rows shape: (167, 140)
next list index out of range
all match rows shape: (48, 140)
2.5 1.84 2.06 -0.5 2.55 1.574
ML : 2.55 3.44 2.88
all match rows shape: (180, 140)
3.5 2.85 1.4 -0.5 2.425 1.525
no alternative odd
no alternative odd
ML : 2.45 3.2 3.0
all match rows shape: (28, 140)
next positional indexers are out-of-bounds
all match rows shape: (104, 140)
2.5 2.01 1.892 -0.5 1.735 2.23
no alternative odd
ML : 1.735 3.86 5.12
all match rows shape: (79, 140)
2.5 1.94 1.88 -0.5 1.68 2.19
ML : 1.67 3.66 4.96
all match rows shape: (134, 140)
2.5 1.99 1.91 -0.5 1.73 2.23
ML : 1.7 3.9 5.1
all match rows shape: (225, 140)
2.5 2.0 1.9 -0.5 1.75 2.2
ML : 1.74 3.7 4.85
all match rows shape: (72, 140)
2.5 2.01 1.892 -0.5 1.735 2.23
no alternative odd
ML : 1.735 3.86 5.12
all match rows shape: (11, 140)
2.5 1.9 1.95 -0.5 1.675 2.15
no alternative odd
no alternative odd
ML : 1.73 3.8 4.75
all match rows shape: (34, 140)
next p

2.5 1.85 2.0 -1.5 1.925 1.925
no alternative odd
no alternative odd
no alternative odd
ML : 1.285 5.5 11.0
all match rows shape: (29, 140)
next positional indexers are out-of-bounds
all match rows shape: (61, 140)
next positional indexers are out-of-bounds
all match rows shape: (119, 140)
next positional indexers are out-of-bounds
all match rows shape: (169, 140)
next positional indexers are out-of-bounds
all match rows shape: (389, 140)
next positional indexers are out-of-bounds
all match rows shape: (61, 140)
2.5 2.56 1.546 0.5 1.645 2.36
ML : 3.45 3.12 2.36
all match rows shape: (32, 140)
2.5 2.53 1.55 0.5 1.64 2.33
ML : 3.45 3.07 2.36
all match rows shape: (122, 140)
3.5 1.689 2.23 -2.5 2.14 1.757
no alternative odd
ML : 1.156 8.13 16.48
all match rows shape: (105, 140)
3.5 1.68 2.16 -2.5 2.19 1.68
no alternative odd
ML : 1.13 7.96 14.46
all match rows shape: (100, 140)
3.5 1.73 2.2 -2.5 2.23 1.73
no alternative odd
ML : 1.16 8.6 15.0
all match rows shape: (253, 140)
3.5 1.72 2.21 

all match rows shape: (235, 140)
2.5 1.9 1.94 -0.5 2.07 1.8
ML : 2.08 3.46 3.51
all match rows shape: (258, 140)
2.5 1.94 1.98 -0.5 2.09 1.86
ML : 2.04 3.45 3.7
all match rows shape: (545, 140)
2.5 1.96 1.96 -0.5 2.11 1.85
ML : 2.11 3.5 3.6
all match rows shape: (124, 140)
2.5 1.934 1.97 -0.5 2.07 1.847
ML : 2.07 3.64 3.67
all match rows shape: (218, 140)
2.5 1.98 1.92 -0.5 2.15 1.675
no alternative odd
ML : 2.2 3.4 3.25
all match rows shape: (58, 140)
next positional indexers are out-of-bounds
all match rows shape: (220, 140)
2.5 2.11 1.8 -0.5 2.05 1.869
ML : 2.05 3.5 3.93
all match rows shape: (2410, 140)
2.5 2.07 1.75 -0.5 1.98 1.86
ML : 1.97 3.16 4.06
all match rows shape: (248, 140)
2.5 2.13 1.78 -0.5 2.04 1.88
ML : 2.02 3.3 3.95
all match rows shape: (827, 140)
2.5 2.17 1.75 -0.5 2.05 1.87
ML : 2.04 3.25 4.0
all match rows shape: (156, 140)
2.5 2.11 1.8 -0.5 2.05 1.869
ML : 2.05 3.5 3.93
all match rows shape: (126, 140)
2.5 1.85 2.0 -0.5 1.98 1.95
no alternative odd
no alternativ

no alternative odd
ML : 1.6569999999999998 3.99 5.41
all match rows shape: (8, 140)
2.5 1.85 1.95 -0.5 1.6 2.3
no alternative odd
no alternative odd
ML : 1.57 4.0 5.5
all match rows shape: (9, 140)
2.5 1.84 2.02 -0.5 1.77 2.11
no alternative odd
ML : 1.78 3.9 4.53
all match rows shape: (24, 140)
2.5 2.46 1.584 0.5 1.555 2.57
ML : 3.23 2.94 2.57
all match rows shape: (25, 140)
next list index out of range
all match rows shape: (64, 140)
next list index out of range
all match rows shape: (54, 140)
next list index out of range
all match rows shape: (22, 140)
2.5 2.46 1.584 0.5 1.555 2.57
ML : 3.23 2.94 2.57
all match rows shape: (17, 140)
2.5 2.375 1.55 -0.5 2.85 1.4
no alternative odd
ML : 3.0 3.0 2.5
all match rows shape: (12, 140)
next positional indexers are out-of-bounds
all match rows shape: (27, 140)
3.5 2.84 1.442 -1.5 2.22 1.704
no alternative odd
ML : 1.393 4.93 7.7
all match rows shape: (25, 140)
2.5 1.68 2.16 -1.5 2.21 1.66
no alternative odd
ML : 1.4 4.46 6.16
all match rows 

no alternative odd
ML : 1.48 4.88 6.3
all match rows shape: (71, 140)
2.5 1.917 1.99 0.5 2.02 1.892
ML : 4.26 3.71 1.892
all match rows shape: (93, 140)
2.5 1.84 1.98 0.5 1.96 1.88
ML : 3.96 3.56 1.87
all match rows shape: (138, 140)
2.5 1.91 1.99 0.5 2.02 1.9
ML : 4.1 3.65 1.89
all match rows shape: (715, 140)
2.5 1.89 2.01 0.5 2.03 1.89
ML : 3.95 3.8 1.88
all match rows shape: (66, 140)
2.5 1.917 1.99 0.5 2.02 1.892
ML : 4.26 3.71 1.892
all match rows shape: (43, 140)
2.5 1.9 1.95 0.5 2.03 1.87
no alternative odd
ML : 4.1 3.75 1.85
all match rows shape: (47, 140)
2.5 1.89 1.96 0.5 1.99 1.86
ML : 4.2 3.73 1.87



In [80]:
# total errors
for i in range(0, 7):
    book = 'bookmaker' + str(i)
    if book not in total_errors.keys():
        continue
    
    print(book)
    print('all total errors')
    print("min error:",  np.array(total_errors[book]).min())
    print("max error:",  np.array(total_errors[book]).max())
    print("mean error:",  np.array(total_errors[book]).mean())
    print("std error:",  np.array(total_errors[book]).std())
    print("25% error:",  np.percentile(total_errors[book], q=25))
    print("50% error:",  np.percentile(total_errors[book], q=50))
    print("75% error:",  np.percentile(total_errors[book], q=75))
    print("95% error:",  np.percentile(total_errors[book], q=95))
    print("99% error:",  np.percentile(total_errors[book], q=99))
    print()
    
    print('int total errors')
    print("min error:",  np.array(int_total_errors[book]).min())
    print("max error:",  np.array(int_total_errors[book]).max())
    print("mean error:",  np.array(int_total_errors[book]).mean())
    print("std error:",  np.array(int_total_errors[book]).std())
    print("25% error:",  np.percentile(int_total_errors[book], q=25))
    print("50% error:",  np.percentile(int_total_errors[book], q=50))
    print("75% error:",  np.percentile(int_total_errors[book], q=75))
    print("95% error:",  np.percentile(int_total_errors[book], q=95))
    print("99% error:",  np.percentile(int_total_errors[book], q=99))
    print()
    
    print('half total errors')
    print("min error:",  np.array(half_total_errors[book]).min())
    print("max error:",  np.array(half_total_errors[book]).max())
    print("mean error:",  np.array(half_total_errors[book]).mean())
    print("std error:",  np.array(half_total_errors[book]).std())
    print("25% error:",  np.percentile(half_total_errors[book], q=25))
    print("50% error:",  np.percentile(half_total_errors[book], q=50))
    print("75% error:",  np.percentile(half_total_errors[book], q=75))
    print("95% error:",  np.percentile(half_total_errors[book], q=95))
    print("99% error:",  np.percentile(half_total_errors[book], q=99))
    print()
    
    print('quarter total errors')
    print("min error:",  np.array(quarter_total_errors[book]).min())
    print("max error:",  np.array(quarter_total_errors[book]).max())
    print("mean error:",  np.array(quarter_total_errors[book]).mean())
    print("std error:",  np.array(quarter_total_errors[book]).std())
    print("25% error:",  np.percentile(quarter_total_errors[book], q=25))
    print("50% error:",  np.percentile(quarter_total_errors[book], q=50))
    print("75% error:",  np.percentile(quarter_total_errors[book], q=75))
    print("95% error:",  np.percentile(quarter_total_errors[book], q=95))
    print("99% error:",  np.percentile(quarter_total_errors[book], q=99))
    print()

bookmaker0
all total errors
min error: 0.0
max error: 0.11690884904170784
mean error: 0.005460710171911101
std error: 0.009142804989785855
25% error: 0.0007630589856083558
50% error: 0.0027440352908878474
75% error: 0.006401154777245566
95% error: 0.019614005529371287
99% error: 0.04928709562260165

int total errors
min error: 1.6916496209939957e-06
max error: 0.11469046773410707
mean error: 0.007226187777898446
std error: 0.012174341930991841
25% error: 0.0016939725878455242
50% error: 0.00377912858114543
75% error: 0.007324206918565668
95% error: 0.026842555551707246
99% error: 0.06879526059097081

half total errors
min error: 0.0
max error: 0.05317584941359077
mean error: 0.004876015329368697
std error: 0.00609350492166771
25% error: 1.2006360753047396e-05
50% error: 0.0029470336641343464
75% error: 0.007167292514780599
95% error: 0.0165880034134365
99% error: 0.028179688292303606

quarter total errors
min error: 2.0386851724563115e-06
max error: 0.11690884904170784
mean error: 0.00

In [83]:
# total home errors
for i in range(0, 7):
    book = 'bookmaker' + str(i)
    if book not in total_1_errors.keys():
        continue
    
    print(book)
    print('all total home errors')
    print("min error:",  np.array(total_1_errors[book]).min())
    print("max error:",  np.array(total_1_errors[book]).max())
    print("mean error:",  np.array(total_1_errors[book]).mean())
    print("std error:",  np.array(total_1_errors[book]).std())
    print("25% error:",  np.percentile(total_1_errors[book], q=25))
    print("50% error:",  np.percentile(total_1_errors[book], q=50))
    print("75% error:",  np.percentile(total_1_errors[book], q=75))
    print("95% error:",  np.percentile(total_1_errors[book], q=95))
    print("99% error:",  np.percentile(total_1_errors[book], q=99))
    print()
    
    try:
        print('int total home errors')
        print("min error:",  np.array(int_total_1_errors[book]).min())
        print("max error:",  np.array(int_total_1_errors[book]).max())
        print("mean error:",  np.array(int_total_1_errors[book]).mean())
        print("std error:",  np.array(int_total_1_errors[book]).std())
        print("25% error:",  np.percentile(int_total_1_errors[book], q=25))
        print("50% error:",  np.percentile(int_total_1_errors[book], q=50))
        print("75% error:",  np.percentile(int_total_1_errors[book], q=75))
        print("95% error:",  np.percentile(int_total_1_errors[book], q=95))
        print("99% error:",  np.percentile(int_total_1_errors[book], q=99))
        print()
    except:
        pass
    
    print('half total home errors')
    print("min error:",  np.array(half_total_1_errors[book]).min())
    print("max error:",  np.array(half_total_1_errors[book]).max())
    print("mean error:",  np.array(half_total_1_errors[book]).mean())
    print("std error:",  np.array(half_total_1_errors[book]).std())
    print("25% error:",  np.percentile(half_total_1_errors[book], q=25))
    print("50% error:",  np.percentile(half_total_1_errors[book], q=50))
    print("75% error:",  np.percentile(half_total_1_errors[book], q=75))
    print("95% error:",  np.percentile(half_total_1_errors[book], q=95))
    print("99% error:",  np.percentile(half_total_1_errors[book], q=99))
    print()
    
    try:
        print('quarter total home errors')
        print("min error:",  np.array(quarter_total_1_errors[book]).min())
        print("max error:",  np.array(quarter_total_1_errors[book]).max())
        print("mean error:",  np.array(quarter_total_1_errors[book]).mean())
        print("std error:",  np.array(quarter_total_1_errors[book]).std())
        print("25% error:",  np.percentile(quarter_total_1_errors[book], q=25))
        print("50% error:",  np.percentile(quarter_total_1_errors[book], q=50))
        print("75% error:",  np.percentile(quarter_total_1_errors[book], q=75))
        print("95% error:",  np.percentile(quarter_total_1_errors[book], q=95))
        print("99% error:",  np.percentile(quarter_total_1_errors[book], q=99))
        print()
    except:
        pass

bookmaker4
all total home errors
min error: 1.5605712026078855e-05
max error: 0.08147888849688645
mean error: 0.009815900989922084
std error: 0.008143556407729501
25% error: 0.003632566065299825
50% error: 0.008230718439435236
75% error: 0.013620037759113357
95% error: 0.024646352617504502
99% error: 0.03407861388793111

int total home errors
min error: 1.5605712026078855e-05
max error: 0.04035899832376666
mean error: 0.011946876352158919
std error: 0.00841763453122075
25% error: 0.005327690508712013
50% error: 0.010837592203702229
75% error: 0.017493417553883672
95% error: 0.02780760917249884
99% error: 0.03616156786937249

half total home errors
min error: 3.049366189233904e-05
max error: 0.08147888849688645
mean error: 0.008104461402125625
std error: 0.007489727447408159
25% error: 0.0032424931785017402
50% error: 0.006518386649818186
75% error: 0.011339702134765456
95% error: 0.01811640364532028
99% error: 0.029701063354524368

quarter total home errors
bookmaker6
all total home er

In [90]:
# total away errors
for i in range(0, 7):
    book = 'bookmaker' + str(i)
    if book not in total_2_errors.keys():
        continue
    
    print(book)
    print('all total away errors')
    print("min error:",  np.array(total_2_errors[book]).min())
    print("max error:",  np.array(total_2_errors[book]).max())
    print("mean error:",  np.array(total_2_errors[book]).mean())
    print("std error:",  np.array(total_2_errors[book]).std())
    print("25% error:",  np.percentile(total_2_errors[book], q=25))
    print("50% error:",  np.percentile(total_2_errors[book], q=50))
    print("75% error:",  np.percentile(total_2_errors[book], q=75))
    print("95% error:",  np.percentile(total_2_errors[book], q=95))
    print("99% error:",  np.percentile(total_2_errors[book], q=99))
    print()
    
    try:
        print('int total away errors')
        print("min error:",  np.array(int_total_2_errors[book]).min())
        print("max error:",  np.array(int_total_2_errors[book]).max())
        print("mean error:",  np.array(int_total_2_errors[book]).mean())
        print("std error:",  np.array(int_total_2_errors[book]).std())
        print("25% error:",  np.percentile(int_total_2_errors[book], q=25))
        print("50% error:",  np.percentile(int_total_2_errors[book], q=50))
        print("75% error:",  np.percentile(int_total_2_errors[book], q=75))
        print("95% error:",  np.percentile(int_total_2_errors[book], q=95))
        print("99% error:",  np.percentile(int_total_2_errors[book], q=99))
        print()
    except:
        pass
    
    print('half total away errors')
    print("min error:",  np.array(half_total_2_errors[book]).min())
    print("max error:",  np.array(half_total_2_errors[book]).max())
    print("mean error:",  np.array(half_total_2_errors[book]).mean())
    print("std error:",  np.array(half_total_2_errors[book]).std())
    print("25% error:",  np.percentile(half_total_2_errors[book], q=25))
    print("50% error:",  np.percentile(half_total_2_errors[book], q=50))
    print("75% error:",  np.percentile(half_total_2_errors[book], q=75))
    print("95% error:",  np.percentile(half_total_2_errors[book], q=95))
    print("99% error:",  np.percentile(half_total_2_errors[book], q=99))
    print()
    
    try:
        print('quarter total away errors')
        print("min error:",  np.array(quarter_total_2_errors[book]).min())
        print("max error:",  np.array(quarter_total_2_errors[book]).max())
        print("mean error:",  np.array(quarter_total_2_errors[book]).mean())
        print("std error:",  np.array(quarter_total_2_errors[book]).std())
        print("25% error:",  np.percentile(quarter_total_2_errors[book], q=25))
        print("50% error:",  np.percentile(quarter_total_2_errors[book], q=50))
        print("75% error:",  np.percentile(quarter_total_2_errors[book], q=75))
        print("95% error:",  np.percentile(quarter_total_2_errors[book], q=95))
        print("99% error:",  np.percentile(quarter_total_2_errors[book], q=99))
        print()
    except:
        pass

bookmaker4
all total away errors
min error: 5.191565700568557e-05
max error: 0.05521655145925586
mean error: 0.008918888918833297
std error: 0.0072234441489936446
25% error: 0.003665946878207621
50% error: 0.007090244438568538
75% error: 0.012975574803117451
95% error: 0.0222275624838697
99% error: 0.03311334406197994

int total away errors
min error: 5.762211005200246e-05
max error: 0.05521655145925586
mean error: 0.009942312883049857
std error: 0.008114951331916701
25% error: 0.003801090062204193
50% error: 0.008117813859475742
75% error: 0.014233682088244132
95% error: 0.025142010449394056
99% error: 0.03510949913011098

half total away errors
min error: 5.191565700568557e-05
max error: 0.037731504412700345
mean error: 0.007968072735861831
std error: 0.0061316271550068135
25% error: 0.00353342564213828
50% error: 0.006459663040516372
75% error: 0.011825930930249307
95% error: 0.01855112376764341
99% error: 0.026285920927369595

quarter total away errors
bookmaker6
all total away err

In [89]:
# handicap errors
for i in range(0, 7):
    book = 'bookmaker' + str(i)
    if book not in hdp_1_errors.keys():
        continue
    
    print(book)
    print('all hdp errors')
    print("min error:",  np.array(hdp_1_errors[book]).min())
    print("max error:",  np.array(hdp_1_errors[book]).max())
    print("mean error:",  np.array(hdp_1_errors[book]).mean())
    print("std error:",  np.array(hdp_1_errors[book]).std())
    print("25% error:",  np.percentile(hdp_1_errors[book], q=25))
    print("50% error:",  np.percentile(hdp_1_errors[book], q=50))
    print("75% error:",  np.percentile(hdp_1_errors[book], q=75))
    print("95% error:",  np.percentile(hdp_1_errors[book], q=95))
    print("99% error:",  np.percentile(hdp_1_errors[book], q=99))
    print()
    
    try:
        print('int hdp errors')
        print("min error:",  np.array(int_hdp_1_errors[book]).min())
        print("max error:",  np.array(int_hdp_1_errors[book]).max())
        print("mean error:",  np.array(int_hdp_1_errors[book]).mean())
        print("std error:",  np.array(int_hdp_1_errors[book]).std())
        print("25% error:",  np.percentile(int_hdp_1_errors[book], q=25))
        print("50% error:",  np.percentile(int_hdp_1_errors[book], q=50))
        print("75% error:",  np.percentile(int_hdp_1_errors[book], q=75))
        print("95% error:",  np.percentile(int_hdp_1_errors[book], q=95))
        print("99% error:",  np.percentile(int_hdp_1_errors[book], q=99))
        print()
    except:
        pass
    
    print('half hdp errors')
    print("min error:",  np.array(half_hdp_1_errors[book]).min())
    print("max error:",  np.array(half_hdp_1_errors[book]).max())
    print("mean error:",  np.array(half_hdp_1_errors[book]).mean())
    print("std error:",  np.array(half_hdp_1_errors[book]).std())
    print("25% error:",  np.percentile(half_hdp_1_errors[book], q=25))
    print("50% error:",  np.percentile(half_hdp_1_errors[book], q=50))
    print("75% error:",  np.percentile(half_hdp_1_errors[book], q=75))
    print("95% error:",  np.percentile(half_hdp_1_errors[book], q=95))
    print("99% error:",  np.percentile(half_hdp_1_errors[book], q=99))
    print()
    
    try:
        print('quarter hdp errors')
        print("min error:",  np.array(quarter_hdp_1_errors[book]).min())
        print("max error:",  np.array(quarter_hdp_1_errors[book]).max())
        print("mean error:",  np.array(quarter_hdp_1_errors[book]).mean())
        print("std error:",  np.array(quarter_hdp_1_errors[book]).std())
        print("25% error:",  np.percentile(quarter_hdp_1_errors[book], q=25))
        print("50% error:",  np.percentile(quarter_hdp_1_errors[book], q=50))
        print("75% error:",  np.percentile(quarter_hdp_1_errors[book], q=75))
        print("95% error:",  np.percentile(quarter_hdp_1_errors[book], q=95))
        print("99% error:",  np.percentile(quarter_hdp_1_errors[book], q=99))
        print()
    except:
        pass

bookmaker0
all hdp errors
min error: 0.0
max error: 0.22651508323653233
mean error: 0.010931017782052204
std error: 0.013670228520173742
25% error: 0.0019241449658968235
50% error: 0.0068746994747648504
75% error: 0.015696276684153465
95% error: 0.034057543263913456
99% error: 0.059339120078327576

int hdp errors
min error: 6.462564288600703e-06
max error: 0.005812472024922688
mean error: 0.0029946547698138093
std error: 0.0020685522439924412
25% error: 0.0013175276825644344
50% error: 0.0038647027444832505
75% error: 0.003972108832810073
95% error: 0.005444399386500164
99% error: 0.005738857497238184

half hdp errors
min error: 1.306816508894748e-07
max error: 0.06917047673489143
mean error: 0.01459336621179274
std error: 0.013928895334945589
25% error: 0.0033218585759885177
50% error: 0.010161152456748007
75% error: 0.02273167475187643
95% error: 0.04169858882669408
99% error: 0.05317222927791447

quarter hdp errors
min error: 0.0
max error: 0.22651508323653233
mean error: 0.01051920

In [88]:
# ml errors
for i in range(0, 7):
    book = 'bookmaker' + str(i)
    if book not in ml_errors.keys():
        continue
    
    print(book)
    print('ml errors')
    print("min error:",  np.array(ml_errors[book]).min())
    print("max error:",  np.array(ml_errors[book]).max())
    print("mean error:",  np.array(ml_errors[book]).mean())
    print("std error:",  np.array(ml_errors[book]).std())
    print("25% error:",  np.percentile(ml_errors[book], q=25))
    print("50% error:",  np.percentile(ml_errors[book], q=50))
    print("75% error:",  np.percentile(ml_errors[book], q=75))
    print("95% error:",  np.percentile(ml_errors[book], q=95))
    print("99% error:",  np.percentile(ml_errors[book], q=99))
    print()

bookmaker0
ml errors
min error: 3.402819893083553e-07
max error: 0.10655407480115969
mean error: 0.014009472777390555
std error: 0.012439923340711924
25% error: 0.003500427381990273
50% error: 0.011210651868286837
75% error: 0.021230904264581713
95% error: 0.03847989354621849
99% error: 0.05100124194299714

bookmaker1
ml errors
min error: 1.2964733213216562e-05
max error: 0.07916762590779991
mean error: 0.015460539296736298
std error: 0.010892872812177325
25% error: 0.007285233930193133
50% error: 0.01294576013285445
75% error: 0.022070488227384978
95% error: 0.035715288621333344
99% error: 0.04551020843478676

bookmaker2
ml errors
min error: 7.0489831923703505e-06
max error: 0.06653625388427475
mean error: 0.014369884775009804
std error: 0.009677473117939038
25% error: 0.007030600493028727
50% error: 0.01283692995923652
75% error: 0.020733111342015553
95% error: 0.030912573061205907
99% error: 0.04041858686372869

bookmaker3
ml errors
min error: 6.656581053032795e-07
max error: 0.0654